In [1]:
import sys
import re
import string
from pandas import read_csv
from itertools import groupby
from operator import itemgetter
from collections import Iterable
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import NE_candidate_module as ne


In [2]:
cachedStopWords = set(stopwords.words("english"))
cachedStopWords.update(("and","make","oh","via","i","i'll","someone","somebody","didn't","i've","they're","we're","we'll","we've","again","thats","that's","i'm","a","so","except","arn't","this","when","it","it's","he's","she's","she'd","he'd","many","can't","cant","yes","no","these","to","may","maybe","<hashtag>","<hashtag>."))
cachedTitles = ["Mr.","Mr","Mrs.","Mrs","Miss","Ms","Sen."]
prep_list=["in","at","of","on","and"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]

In [3]:
def printList(mylist):
    print("["),
    #print "[",
    for item in mylist:
        if item != None:
            if isinstance(item,ne.NE_candidate):
                item.print_obj()
                #print (item.phraseText)
            else:
                print (item+",", end="")
                #print item+",",
    #print "]"
    print("]")

In [4]:
def flatten(mylist, outlist,ignore_types=(str, bytes, ne.NE_candidate)):
    
    if mylist !=[]:
        for item in mylist:
            #print not isinstance(item, ne.NE_candidate)
            if isinstance(item, list) and not isinstance(item, ignore_types):
                flatten(item, outlist)
            else:
                if isinstance(item,ne.NE_candidate):
                    item.phraseText=item.phraseText.strip(' \t\n\r')
                    item.reset_length()
                else:
                    item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist

In [5]:
def consecutive_cap(tweetWordList_cappos,tweetWordList):
    output=[]
    #identifies consecutive numbers in the sequence
    for k, g in groupby(enumerate(tweetWordList_cappos), lambda element: element[0]-element[1]):
        output.append(list(map(itemgetter(1), g)))
    count=0
    if output:        
        final_output=[output[0]]
        for first, second in (zip(output,output[1:])):
            if ((second[0]-first[-1])==2) & (tweetWordList[first[-1]+1] in prep_list):
                (final_output[-1]).extend([first[-1]+1]+second)
            else:
                final_output.append(second)
                #merge_positions.append(False)
    else:
        final_output=[]
    
    return final_output

In [6]:
#basically splitting the original NE_candidate text and building individual object from each text snippet
def build_custom_NE(phrase,pos,prototype,feature_index,feature_value):
    #print("Enters")
    position=pos
    custom_NE= ne.NE_candidate(phrase,position)
    for i in range(9):
        custom_NE.set_feature(i,prototype.features[i])
    custom_NE.set_feature(feature_index,feature_value)
    if (feature_index== ne.is_csl) & (feature_value== True):
        custom_NE.set_feature(ne.start_of_sentence, False)
    return custom_NE

In [7]:
def abbrv_algo(ne_element):
    '''abbreviation algorithm 
    trailing apostrophe:
           |period:
           |     multiple letter-period sequence:
           |         all caps
           | non period:
           |     ?/! else drop apostrophe
    else:
        unchanged
    '''
    phrase= ne_element.phraseText
    #print("=>"+phrase)
    #since no further split occurs we can set remaining features now
    ne_element.set_feature(ne.capitalized, True)
    if ne_element.phraseText.isupper():
        ne_element.set_feature(ne.all_capitalized, True)
    else:
        ne_element.set_feature(ne.all_capitalized, False)
        
    abbreviation_flag=False
    p=re.compile(r'[^a-zA-Z\d\s]$')
    match_list = p.findall(phrase)
    if len(match_list)>0:
        #print("Here")
        if phrase.endswith('.'):
            p1= re.compile(r'([a-zA-Z][\.]\s*)')
            match_list = p1.findall(phrase)
            if ((len(match_list)>1) & (len(phrase)<6)):
                #print ("1. Found abbreviation: "+phrase)
                abbreviation_flag= True
            else:
                phrase= phrase[:-1]
        else:
            phrase= phrase[:-1]
            #if not (phrase.endswith('?')|phrase.endswith('!')|phrase.endswith(')')|phrase.endswith('>')):
                #phrase= phrase[:-1]
    else:
        p2=re.compile(r'([^a-zA-Z0-9_\s])')
        match_list = p2.findall(phrase)
        if ((len(match_list)==0) & (phrase.isupper()) & (len(phrase)<7)& (len(phrase)>1)):
            #print ("2. Found abbreviation!!: "+phrase)
            abbreviation_flag= True
        else:
            #print("Here-> "+phrase)
            p3= re.compile(r'([A-Z][.][A-Z])')
            p4= re.compile(r'\s')
            match_list = p3.findall(phrase)
            match_list1 = p4.findall(phrase)
            if ((len(match_list)>0) & (len(match_list1)==0)):
                abbreviation_flag= True
                print ("3. Found abbreviation!!: "+phrase)
            
    #element= ne.NE_candidate(phrase.strip())
    ne_element.phraseText=phrase
    ne_element.reset_length()
    ne_element.set_feature(ne.abbreviation, abbreviation_flag)
    return ne_element
    

In [8]:
def punct_clause(NE_phrase_in):
    
    NE_phrases=entity_info_check(NE_phrase_in)
    cap_phrases=NE_phrases.phraseText.strip()
    #print (cap_phrases)
    if (re.compile(r'[^a-zA-Z0-9_\s]')).findall(cap_phrases):
        #case of intermediate punctuations: handles abbreviations
        p1= re.compile(r'(?:[a-zA-Z0-9][^a-zA-Z0-9_\s]\s*)')
        match_lst = p1.findall(cap_phrases)
        if match_lst:
            index= (list( p1.finditer(cap_phrases) )[-1]).span()[1]
        
        p= re.compile(r'[^a-zA-Z\d\s]')
        match_list = p.findall(cap_phrases)

        p2=re.compile(r'[^a-zA-Z\d\s]$') #ends with punctuation

        if len(match_list)-len(match_lst)>0:
            if (p2.findall(cap_phrases)):
                #only strips trailing punctuations, not intermediate ones following letters
                cap_phrases = cap_phrases[0:index]+re.sub(p, '', cap_phrases[index:])
                NE_phrases.phraseText= cap_phrases
        
    
    #comma separated NEs
    #lst=filter(lambda(word): word!="", re.split('[,]', cap_phrases))
    start_of_sentence_fix=NE_phrases.features[ne.start_of_sentence]
    wordlst=list(filter(lambda word: word!="", re.split('[,]', cap_phrases)))
    if (NE_phrases.features[ne.date_indicator]==False) & (len(wordlst)>1):
        pos=NE_phrases.position
        combined=[]
        prev=0
        for i in range(len(wordlst)):
            word=wordlst[i]
            word_len=len(list(filter(lambda individual_word: individual_word!="", re.split('[ ]', word))))
            word_pos=pos[(prev):(prev+word_len)]
            prev=prev+word_len
            combined+=[[word]+word_pos]
        
        lst_nsw=list(filter(lambda element: (((str(element[0])).lower() not in cachedStopWords) & (len(str(element[0]))>1)) ,combined))
        #print (lst_nsw)
        final_lst= list(map(lambda element:build_custom_NE(str(element[0]),element[1:],NE_phrases,ne.is_csl,True), lst_nsw))
        final_lst[0].set_feature(ne.start_of_sentence, NE_phrases.features[ne.start_of_sentence])
    else:
        NE_phrases.set_feature(ne.is_csl,False)
        final_lst=[NE_phrases]
    
    #check abbreviation
    final_lst= list(map(lambda phrase: abbrv_algo(phrase), final_lst))

    
    #print(lst)
    return final_lst

In [9]:
def title_check(ne_phrase):
    title_flag=False
    words=ne_phrase.phraseText.split()
    for word in words:
        if word in cachedTitles:
            title_flag= True
            break
    ne_phrase.set_feature(ne.title,title_flag)
    return ne_phrase

In [10]:
def entity_info_check(ne_phrase):
    flag1=False #has number
    flag2=False
    flag3=False
    flag_ind=[] #is number
    month_ind=[]
    words=ne_phrase.phraseText.split()
    for word in words:
        word=(word.strip()).rstrip(string.punctuation).lower()
        if not word.isalpha():
            flag_ind+=[True]
        else:
            flag_ind+=[False]
            if word in month_list:
                month_ind+=[True]
    if True in flag_ind:
        flag1=True
    if True in month_ind:
        flag3=True
    if not (False in flag_ind):
        flag2=True
    ne_phrase.set_feature(ne.has_number,flag1)
    ne_phrase.set_feature(ne.is_number,flag2)
    ne_phrase.set_feature(ne.date_indicator,flag3)
    return ne_phrase

In [11]:
#%%timeit -o
def f(x,tweetWordList):

    #list1=map(lambda word: check(tweetWordList[word], word), x)
    list1=list(map(lambda word: tweetWordList[word]+" ", x[:-1]))
    phrase="".join(list1)+tweetWordList[x[-1]]
    '''if phrase=="":
        print "JUST_DIGIT_ERROR"
        phrase="JUST_DIGIT_ERROR"'''
    if not (phrase[0].isdigit() & len(x)==1):
        NE_phrase= ne.NE_candidate(phrase.strip(),x)
        if 0 in x:
            NE_phrase.set_feature(ne.start_of_sentence,True)
        else:
            NE_phrase.set_feature(ne.start_of_sentence,False)
    else:
        NE_phrase= ne.NE_candidate("JUST_DIGIT_ERROR",[])
        
    return NE_phrase

In [12]:
def capCheck(word):
    if word.startswith('@'):
        return False
    elif "<Hashtag" in word:
        return False
    elif word.lower() in cachedStopWords:
        return False
    elif word[0].isdigit():
        return True
    else:
        p=re.compile(r'^[\W]*[A-Z]')
        l= p.match(word)
        if l:
            return True
        else:
            return False

In [13]:
'''
def final_split(ne_pctd):
    print("++ "+ne_pctd.phraseText)
    final=[]
    splitTextList=[]
    str_pattern=re.compile('[.:?!]+[\s]+')
    if ((ne_pctd.features[ne.abbreviation] is False)&(ne_pctd.features[ne.title] is False)):
        splitTextList=list(filter (lambda sentence: len(sentence.strip())>0, str_pattern.split(ne_pctd.phraseText)))
    if len(splitTextList)<2:
        return [ne_pctd]
    else:
        #print("Had to split")
        splitTextList_nsw=list(filter(lambda phrase: (phrase.lower() not in cachedStopWords) & (len(phrase)>1) ,splitTextList))
        for text in splitTextList_nsw:
            split_ne= ne.NE_candidate(text.strip())
            split_ne.set_feature(ne.capitalized,True)
            split_ne_pc=punct_clause(split_ne)
            split_ne_pctd=list(map(lambda element: title_check(element), split_ne_pc))
            final+=split_ne_pctd
        return final
    '''

'\ndef final_split(ne_pctd):\n    print("++ "+ne_pctd.phraseText)\n    final=[]\n    splitTextList=[]\n    str_pattern=re.compile(\'[.:?!]+[\\s]+\')\n    if ((ne_pctd.features[ne.abbreviation] is False)&(ne_pctd.features[ne.title] is False)):\n        splitTextList=list(filter (lambda sentence: len(sentence.strip())>0, str_pattern.split(ne_pctd.phraseText)))\n    if len(splitTextList)<2:\n        return [ne_pctd]\n    else:\n        #print("Had to split")\n        splitTextList_nsw=list(filter(lambda phrase: (phrase.lower() not in cachedStopWords) & (len(phrase)>1) ,splitTextList))\n        for text in splitTextList_nsw:\n            split_ne= ne.NE_candidate(text.strip())\n            split_ne.set_feature(ne.capitalized,True)\n            split_ne_pc=punct_clause(split_ne)\n            split_ne_pctd=list(map(lambda element: title_check(element), split_ne_pc))\n            final+=split_ne_pctd\n        return final\n    '

In [14]:
def trueEntity_process(tweetWordList_cappos,tweetWordList):
    
    #returns list with position of consecutively capitalized words
    output = consecutive_cap(tweetWordList_cappos,tweetWordList)
    #print ("=>"+str(output))

    #consecutive capitalized phrases 
    consecutive_cap_phrases1=list(map(lambda x: f(x,tweetWordList), output))

    consecutive_cap_phrases=list(filter(lambda candidate:(candidate.phraseText!="JUST_DIGIT_ERROR"),consecutive_cap_phrases1))

    
    #implement the punctuation clause
    ne_List_pc=flatten(list(map(lambda NE_phrase: punct_clause(NE_phrase), consecutive_cap_phrases)),[])
    

    #implement title detection
    ne_List_allCheck= list(map(lambda element: title_check(element), ne_List_pc))
    
    #implement number check
    #ne_List_allCheck= list(map(lambda element: number_check(element), ne_List_pctd))
    
    return ne_List_allCheck

In [15]:

'''This is the main module. I am not explicitly writing it as a function as I am not sure what argument you are 
passing.However you can call this whole cell as a function and it will call the rest of the functions in my module
to extract candidates and features
'''

'''#reads input from the database file and converts to a dataframe. You can change this part accordingly and
#directly convert argument tuple to the dataframe'''

#Collection.csv
df = read_csv('/home/satadisha/Desktop/GitProjects/ELTweetTracker/500Sample.csv', index_col='ID', header=0, encoding='utf-8')
#print (df.columns.values.tolist())

#%%timeit -o
#module_capital_punct.main:
'''I am running this for 100 iterations for testing purposes. Of course you no longer need this for loop as you are
#running one tuple at a time'''

count=0
ne_count=0
userMention_count=0
token_count=0

NE_list_phase1=[]
UserMention_list=[]
for index, row in df.iterrows():
    
    if count<100:
        #tweetText=unicode(row['Tweets']).encode('utf-8')
        userType=str(row['User Type'])
        tweetText=str(row['TweetText'])
        #tweetText=row['Tweets']
        print(userType+": "+tweetText)
        #capitalization module
        #if all words are capitalized:
        if tweetText.isupper():
            #print "All caps module"
            print ("All caps module")
        elif tweetText.islower():
            #print "All caps module"
            print ("All lower module")
        else:
            ne_List_final=[]
            userMention_List_final=[]
            #pre-modification: returns word list split at whitespaces; retains punctuation
            #tweetSentenceList=list(filter (lambda sentence: len(sentence)>0, tweetText.split('\n')))
            tweetSentenceList=sent_tokenize(tweetText)
            
            #printList(tweetSentenceList)
            for sentence in tweetSentenceList:
                tweetWordList= sentence.split()
                
                token_count+=len(tweetWordList)
                #print (tweetWordList)
                #returns position of words that are capitalized
                tweetWordList_cappos = list(map(lambda element : element[0], filter(lambda element : capCheck(element[1]), enumerate(tweetWordList))))
                
                #returns list of @userMentions
                userMentionswPunct=list(filter(lambda phrase: phrase.startswith('@'), tweetWordList))
                userMentions=list(map(lambda mention: mention.rstrip(string.punctuation), userMentionswPunct))
                
                #non @usermentions are processed in this function to find non @, non hashtag Entities
                ne_List_allCheck= trueEntity_process(tweetWordList_cappos,tweetWordList)
                
                #function to process and store @ user mentions
                
                ne_count+=len(ne_List_allCheck)
                userMention_count+=len(userMentions)
                
                ne_List_final+=ne_List_allCheck
                userMention_List_final+=userMentions
                
            printList(ne_List_final)
            if(userMention_List_final):
                print(userMention_List_final)
            
            NE_list_phase1+=ne_List_final
            UserMention_list+=userMention_List_final
            print ("")
        count+=1
    else:
        break
print("Total number of tokens processed: "+str(token_count))
print ("Total number of NEs extracted: "+str(ne_count))


Government: @Brendan62 congrats on your 11th place finish at <Hashtag>. What a story! What a day for @CityOfLasVegas!
[
]
['@Brendan62', '@CityOfLasVegas']

Journalist: Great drive by @Blaney in the <Hashtag>! (Even though I'm sure he's bummed w/ 2nd) That one was survival of the fittest. @Team_Penske
[
Great 1 [0] [] True True False False False False False False False 
(Even 1 [0] [] True True False False False False True True False 
]
['@Blaney', '@Team_Penske']

Journalist: Congratulations to @KurtBusch, who talked about what he had to do in order to win the <Hashtag> earlier this week. 
[
Congratulations 1 [0] [] True True False False False False False False False 
]
['@KurtBusch']

Journalist: Very likely this has never happened in <Hashtag> history. Great start Kurt.
[
Great 1 [0] [] True True False False False False False False False 
Kurt 1 [2] [] True False False False False False False False False 
]

Journalist: Tune-in for the race this Sunday!  I'm excited to be the honora

In [16]:
'''#abbreviation count
f3_count=0
for NE in NE_list:
    if ((NE.features[ne.abbreviation] is False)&(NE.features[ne.title] is False)):
        print(NE.phraseText)
        f3_count+=1
print("Total number of abbreviated NEs: "+str(f3_count))'''

'#abbreviation count\nf3_count=0\nfor NE in NE_list:\n    if ((NE.features[ne.abbreviation] is False)&(NE.features[ne.title] is False)):\n        print(NE.phraseText)\n        f3_count+=1\nprint("Total number of abbreviated NEs: "+str(f3_count))'

In [17]:
'''f3_count=0
for NE in NE_list:
    if (NE.features[ne.title] is True):
        print(NE.phraseText)
        f3_count+=1
print("Total number of abbreviated NEs: "+str(f3_count))'''

'f3_count=0\nfor NE in NE_list:\n    if (NE.features[ne.title] is True):\n        print(NE.phraseText)\n        f3_count+=1\nprint("Total number of abbreviated NEs: "+str(f3_count))'

In [18]:
l=[1,2,3]
print(l[2:3])
word="Feb"
print(word.isalpha())
if word.rstrip(string.punctuation) in month_list:
    print(True)

[3]
True
